In [37]:
import cv2
import imutils

# First cell is adapted from https://www.geeksforgeeks.org/python/pedestrian-detection-using-opencv-python/

In [38]:
# Initializing the HOG person
# detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

cap = cv2.VideoCapture('occlusion.mp4')

boxes = []

images = []

while cap.isOpened():
    # Reading the video stream
    ret, image = cap.read()
    if ret:
        image = imutils.resize(image,
                               width=min(800, image.shape[1]))

        # Detecting all the regions
        # in the Image that has a
        # pedestrian inside it
        (regions, _) = hog.detectMultiScale(image,
                                            winStride=(4, 4),
                                            padding=(4, 4),
                                            scale=1.05)

        if len(regions) > 0:
            regions = regions[:1] # Keep only the first (most confident) detection
        boxes.append(regions)



        # Drawing the regions in the
        # Image
        for (x, y, w, h) in regions:
            cv2.rectangle(image, (x, y),
                          (x + w, y + h),
                          (0, 0, 255), 1)
        height, width, layers = image.shape
        images.append(image)
        # Showing the output Image (commented out for local environment)
        # cv2_imshow(image)

    else:
        break

video = cv2.VideoWriter("occlusion_detections.mp4", cv2.VideoWriter_fourcc(*'MP4V'), 30, (width, height), True)

for image in images:
  video.write(image)

video.release()
cap.release()

In [39]:
print(len(boxes))
boxes = [boxes[i] for i in range(0, len(boxes), 2)] # Only use every-other detection so there is more significant movement
                                                    # between frames
print(len(boxes))

96
48


In [40]:
import numpy as np

# Define the Kalman Filter class
class KF:
    def __init__(self, x, p, q, r, a, h):
        self.xk = x # initial state
        self.P = p  # uncertainty covariance
        self.Q = q  # process noise covariance
        self.R = r  # measurement noise covariance
        self.A = a  # state transition matrix
        self.H = h  # measurement

    def predict(self):
        xk_p = np.dot(self.A, self.xk)
        pk_p = np.dot(np.dot(self.A, self.P), self.A.T) + self.Q
        self.pk_p = pk_p
        self.xk_p = xk_p

    def kalman_gain(self):
        Kk = np.dot(np.dot(self.pk_p, self.H.T), np.linalg.inv(np.dot(np.dot(self.H, self.pk_p), self.H.T) + self.R))
        return Kk

    def estimate(self, zk):
        xk = self.xk_p + np.dot(self.kalman_gain(), (zk - np.dot(self.H, self.xk_p)))
        pk = self.pk_p - np.dot(np.dot(self.kalman_gain(), self.H), self.pk_p)

        # Update
        self.xk = xk
        self.P = pk
        return xk, pk

    def update(self, zk):
        self.predict()
        self.estimate(zk)

In [41]:
# Initialize system parameters
Q = np.eye(8)
R = np.eye(4)
dt = 1
A = np.array([[1, 0, 0, 0, dt, 0, 0, 0], # State transition matrix
             [0, 1, 0, 0, 0, dt, 0, 0],
             [0, 0, 1, 0, 0, 0, dt, 0],
             [0, 0, 0, 1, 0, 0, 0, dt],
             [0, 0, 0, 0, 1, 0, 0, 0],
             [0, 0, 0, 0, 0, 1, 0, 0],
             [0, 0, 0, 0, 0, 0, 1, 0],
             [0, 0, 0, 0, 0, 0, 0, 1]])

H = np.array([[1, 0, 0, 0, 0, 0, 0, 0], # State-to-measurement matrix
     [0, 1, 0, 0, 0, 0, 0, 0],
     [0, 0, 1, 0, 0, 0, 0, 0],
     [0, 0, 0, 1, 0, 0, 0, 0]])

P = np.eye(8)

# The first 4 entries are just the box from the 1st frame
# The last 4 entries are the "rate of change" (difference)
x0 = np.concatenate((boxes[1][0], (boxes[1][0]-boxes[0][0])))

In [42]:
KalmanFilter = KF(x0, P, Q, R, A, H)

In [43]:
kalman_boxes = []
empty_frames = []

for i in range(2, len(boxes)*2):
  if i%2==0 and len(boxes[i//2]) > 0:
    KalmanFilter.update(boxes[i//2][0])
    kalman_boxes.append(KalmanFilter.xk)
  else:
    empty_frames.append(i)
    # If no detection, use Kalman prediction
    KalmanFilter.predict()
    KalmanFilter.update(np.dot(KalmanFilter.xk_p, KalmanFilter.H.T))
    kalman_boxes.append(KalmanFilter.xk)


In [44]:
len(kalman_boxes)

94

Kalman_boxes[i] corresponds to images[i+2]

In [45]:
for i in range(len(kalman_boxes)):

  x, y, w, h = kalman_boxes[i][:4]


  cv2.rectangle(images[i+2], (int(x), int(y)),
                        (int(x + w), int(y + h)),
                        (0, 255, 0), 1)
  # Add kalman bounding boxes to corresponding frames

In [46]:
video = cv2.VideoWriter("occlusion_comparison.mp4", cv2.VideoWriter_fourcc(*'MP4V'), 30, (width, height), True)

for image in images:
  video.write(image)

video.release()